*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split
# split the data into train and test
train_labels, test_labels = train_test_split(dataset,# data
                                                    test_size=0.20, #percent to divide
                                                    random_state=1 # randomice data
                                                   )

In [ ]:
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
# quit the expenses column
train_expenses = train_labels.pop('expenses')
test_expenses = test_labels.pop('expenses')

In [ ]:
train_labels.describe

In [ ]:
#some names
CATEGORICAL_COLUMNS = ['sex','smoker','region']
NUMERIC_COLUMNS = ['age', 'bmi','children']
feature_columns = []

In [ ]:
#create a dictionary for encode
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dataset[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

In [ ]:
#this is a dictionary
feature_columns

In [ ]:
#encode the numeric data
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [ ]:
feature_columns

In [ ]:
train_labels

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=20):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

In [ ]:
# here we will call the input_function that was returned to us to get a dataset object we can feed to the model
train_input_fn = make_input_fn(train_labels, train_expenses)

In [ ]:
# here we make a input function for the data  for this purpose the shuffle is false
# and the num of epochs is one, this in the evaluation inded
eval_input_fn = make_input_fn(test_labels, test_expenses, num_epochs=1, shuffle=False)

In [ ]:
ds = make_input_fn(train_labels, train_expenses, batch_size=10)()
for feature_batch, label_batch in ds.take(1):
  print('Some feature keys:', list(feature_batch.keys()))
  print()
  print('A batch of class:', feature_batch['region'].numpy())
  print()
  print('A batch of Labels:', label_batch.numpy())


In [ ]:
# we can explore some classes of particular column
age_column = feature_columns[4]
tf.keras.layers.DenseFeatures([age_column])(feature_batch).numpy()


In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:

linear_est.train(train_input_fn)

In [ ]:
# We create a linear estimtor by passing the feature columns we created earlier
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((dict(train_labels), expenses_train))

In [ ]:

ds

In [ ]:
# train the model
linear_est.train(train_input_fn) 

In [ ]:
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on testing data

In [ ]:
clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

In [ ]:

#encode the data
dataset['sex'] = dataset['sex'].apply({'male':0, 'female':1}.get)
dataset['smoker'] = dataset['smoker'].apply({'yes':1, 'no':0}.get)
dataset['region'] = dataset['region'].apply({'southwest':1, 'southeast':2, 'northwest':3, 'northeast':4}.get)

In [ ]:
#for check
dataset.head()

In [ ]:
y = dataset.pop('expenses')

In [ ]:
# importing the model
from sklearn.linear_model import LinearRegression

In [ ]:
#make the model
model = LinearRegression()
# Fit linear model 
model.fit(train_labels,expenses_train)

In [ ]:
#predicions
predictions = model.predict(test_labels)

In [ ]:
predictions[1:3]

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
# comparing the results
plt.scatter(expenses_test,predictions) #data test vs predictions
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
# Predict charges for new customer : Name- Evelyn
data = {'age' : 28,
        'sex' : 1,
        'bmi' : 45.50,
        'children' : 0,
        'smoker' : 0,
        'region' : 3}
index = [1]
evelyn_df = pd.DataFrame(data,index)
evelyn_df

In [ ]:
prediction_evelyn = model.predict(evelyn_df)
print("Medical Insurance cost for Evelyn is : ",prediction_evelyn)

In [ ]:
model.


In [ ]:

loss, mae, mse = model.predict(test_dataset, test_labels, verbose=2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
